In [66]:
def findPath(streams):
    '''Return the destination's SLA for every stream in `streams` '''

    path =[]
    for stream in streams :
        for SLA in client_SLAs:
            if stream.addrDest in SLA.clientNetwok :
                path.append(SLA) 

    return path

class Resa :
    def __init__(self, id , streams):
        self.id = id
        self.streams = streams
        self.status = 'Unactive'
        self.path = findPath(streams)
        self.capacity = 0

    def askResa(self, tspec):
        '''Check if all SLAs in path have the capacity, and if possible, book the require capacity'''
        resaOK = True

        # Check if all SLA in path have the capacity
        for SLA in self.path :
            if not SLA.checkCapacity(tspec) :
                resaOK = False 
                break
        # Book require capacity on evry SLA in path
        if resaOK :
            for SLA in self.path :
                SLA.book(self.streams, tspec)
        self.status = 'Active'
        self.capacity += tspec
        return resaOK
    
    def cancelResa(self):
        '''Ask all SLAs in path to unbook the resources allocated to this reservation'''
        for SLA in self.path :
            SLA.unBook(self.streams, self.capacity)
            self.capacity = 0
        self.status = 'Unactive'
    

In [67]:
class Steam :
    def __init__(self,portDest, addrDest, portSrc, addrSrc,protocol='UDP',codec='Default', params=None):
        self.portDest = portDest
        self.addrDest = addrDest
        self.portSrc = portSrc
        self.addrSrc = addrSrc
        self.protocol = protocol
        self.codec = codec
        self.params = params
    

In [68]:
class SLA :
    def __init__(self,id, CE, clientNetwork, capacity): 
        self.id = id
        self.CE = CE
        self.clientNetwork = clientNetwork
        self.capacity = capacity
        self.usage = 0

    def checkCapacity(self ,tspec) :
        '''Check if the SLA have the capacity to book `tspec`'''
        result = False
        if self.capacity > (self.usage + tspec):
            result = True
        return result

    def book(self,streams, tspec):
        '''Upadte SLA's usage and book require `tspec` on the CE '''
        self.usage += tspec
        self.CE.book(streams, tspec)

    def unBook(self, streams, tspec):
        '''Check if the SLA have the capacity to book `tspec`'''
        self.usage -= tspec
        self.CE.unBook(streams)
    

In [69]:
import paramiko

def getBookingConfig(streams, tspec):
    '''Return the list of commands to send to the router to protect all stream in streams acording to tspec'''
    config_commands =[]
    # TO DO !
    return config_commands

def getUnBookingConfig(streams):
    '''Return the list of commands to send to the router to unprotect all stream in streams  '''
    config_commands =[]
    # TO DO !
    return config_commands


class CE :
    def __init__(self,id, ipAddress, username='root', password='7nains'):
        self.id = id
        self.ipAddress = ipAddress
        self.username = username
        self.password = password

    def book(self, streams, tspec) :
        config_commands = getBookingConfig(streams,tspec)

        # initialize the SSH client
        client = paramiko.SSHClient()
        # add to known hosts
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        try:
            client.connect(hostname=self.hostname, username=self.username, password=self.password)
        except:
            print("[!] Cannot connect to the SSH Server")
            exit()

        # execute the commands
        for command in config_commands:
            print("="*50, command, "="*50)
            stdin, stdout, stderr = client.exec_command(command)
            print(stdout.read().decode())
            err = stderr.read().decode()
            if err:
                print(err)
        
        print ("="*50, 'booking over', "="*50)

    def unBook(self, streams) :
        config_commands = getUnBookingConfig(streams)

        # initialize the SSH client
        client = paramiko.SSHClient()
        # add to known hosts
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        try:
            client.connect(hostname=self.hostname, username=self.username, password=self.password)
        except:
            print("[!] Cannot connect to the SSH Server")
            exit()

        # execute the commands
        for command in config_commands:
            print("="*50, command, "="*50)
            stdin, stdout, stderr = client.exec_command(command)
            print(stdout.read().decode())
            err = stderr.read().decode()
            if err:
                print(err)

        print ("="*50, 'unbooking over', "="*50)



In [70]:
from ipaddress import IPv4Network

# Create CEs
CE1= CE ('CE1', '193.168.1.11',)
CE2= CE ('CE2', '193.168.2.22',)
CE3= CE ('CE3', '193.168.3.33',)

# Create SLAs 
SLA1 = SLA(id='SLA1', clientNetwork=IPv4Network('192.168.1.0/24'), CE=CE1, capacity=1000)
SLA2 = SLA(id='SLA2', clientNetwork=IPv4Network('192.168.2.0/24'), CE=CE2, capacity=1000)
SLA3 = SLA(id='SLA3', clientNetwork=IPv4Network('192.168.3.0/24'), CE=CE3, capacity=1000)

client_SLAs=[SLA1,SLA2,SLA3]